### Introducing the ReadMultiModelData class

The `pyaerocom.ReadMultiModelData` class provides an interface to import multiple variables for an arbitrary number of different models and within a specific time interval. It makes large use of the `ReadModelData` class that provides an interface for importing results from a single model.

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from pyaerocom.io.readmodeldata import ReadMultiModelData

Define two models

In [2]:
models = ["AATSR_SU_v4.3", "CAM5.3-Oslo_CTRL2016"]

Create and initiate `ReadMultiModelData` class. The direcory for each model as well as all valid files in the model directories are searched on initiation of the instance. Valid here means, that the files belong to the corresponding model. Subsetting of the time interval is done in a later process.

In [3]:
read = ReadMultiModelData(models, '2010-01-01','2011-12-31', verbose=False)

Let's have a look into one of the two import classes

In [4]:
read_aatsr = read["AATSR_SU_v4.3"]
print("Model ID: %s\n"
      "Model dir: %s\n"
      "First detected file: %s"
      %(read_aatsr.model_id, read_aatsr.model_dir, os.path.basename(read_aatsr.files[0])))

Model ID: AATSR_SU_v4.3
Model dir: /lustre/storeA/project/aerocom/aerocom-users-database/CCI-Aerosol/CCI_AEROSOL_Phase2/AATSR_SU_v4.3/renamed
First detected file: aerocom.AATSR_SU_v4.3.daily.od550erraer.2003.nc


Define a bunch of test variables that are supposed to be loaded for each model (i.e. that must be contained in both input models)

In [5]:
test_vars = ["od550aer", "od550dust"]

Read all variables for all models

In [6]:
result_dict = read.read(test_vars)
for key, val in result_dict.items():
    print("Model ID: %s (type=%s)" %(key, type(val)))

Rolling longitudes to -180 -> 180 definition
Rolling longitudes to -180 -> 180 definition
Model ID: AATSR_SU_v4.3 (type=<class 'pyaerocom.io.readmodeldata.ReadModelData'>)
Model ID: CAM5.3-Oslo_CTRL2016 (type=<class 'pyaerocom.io.readmodeldata.ReadModelData'>)


Print some information about the different data objects

In [7]:
for model_id, result in read.results.items():
    print("Current model: %s" %model_id)
    for var_name, data in result.data.items():
        print("\nCurrent variable: %s" %var_name)
        # data is of type pyaerocom.ModelData which uses an extended representation of the Cube class
        print(repr(data))

Current model: AATSR_SU_v4.3

Current variable: od550aer
pyaerocom.ModelData
Grid data: <iris 'Cube' of atmosphere_optical_thickness_due_to_ambient_aerosol / (1) (time: 730; latitude: 180; longitude: 360)>

Current variable: od550dust
pyaerocom.ModelData
Grid data: <iris 'Cube' of atmosphere_optical_thickness_due_to_ambient_aerosol / (1) (time: 730; latitude: 180; longitude: 360)>
Current model: CAM5.3-Oslo_CTRL2016

Current variable: od550aer
pyaerocom.ModelData
Grid data: <iris 'Cube' of Aerosol optical depth at 550nm / (1) (time: 365; latitude: 192; longitude: 1)>

Current variable: od550dust
pyaerocom.ModelData
Grid data: <iris 'Cube' of mineral aerosol optical depth 550nm / (1) (time: 365; latitude: 192; longitude: 1)>


Now arbitrarily crop the last data object.

In [8]:
dat_crop = data.crop(lon_range=(-30, 30),
                     lat_range=(0, 45),
                     time_range=('2010-03-15','2010-06-22'))

print("\nStart / stop before crop: %s - %s\n"
      %(data.grid.coord("time").cell(0).point,
        data.grid.coord("time").cell(-1).point))

print("Start / stop after crop: %s - %s"
      %(dat_crop.grid.coord("time").cell(0).point,
        dat_crop.grid.coord("time").cell(-1).point))

Cropping along time axis based on Timestamps

Start / stop before crop: 2010-01-01 10:30:00 - 2010-12-31 10:30:00

Start / stop after crop: 2010-03-15 10:30:00 - 2010-06-22 10:30:00
